**Data loader**

In [33]:
import os
import pandas as pd
from urllib import urlopen
import os

decklists_path = 'data/decks/'
matches_path = 'data/matches/'

loader_exists = False
converter_exists = False
saver_exists = False

class Loader():
    def __init__(self):
        loader_exists = True
        self.deckfiles = []
        self.decks = []
        self.cards = []
        self.player_archetype = []
        self.allcards = []
        self.alias_dict = {}
        
    def loadAllDeckFilenames(self):
        if len(self.deckfiles) == 0:
            tmp = os.listdir(decklists_path)
            deckfiles = []
            for i in tmp:
                deckfiles.append(i.replace('~', ''))
            self.deckfiles = list(set(deckfiles))
        
    def loadDecks(self, filename):
        data = pd.read_json('data/decks/'+filename)
        return data['cards']
    
    def loadPlayerArchetype(self, filename):
        data = pd.read_json('data/decks/'+filename)
        tmp = []
        for i in range(0, len(data)):
            tmp.append({'draft': data['draft'][i], 'player': data['player'][i], 'archetype': data['name'][i]})
        return tmp
            
    def loadAllPlayerArchetype(self):
        self.loadAllDeckFilenames()
        for i in self.deckfiles:
            tmp = self.loadPlayerArchetype(i)
            for j in tmp:
                self.player_archetype.append(j)
    
    def loadAllDecks(self):
        self.loadAllDeckFilenames()
        for i in self.deckfiles:
            tmp_decks = self.loadDecks(i)
            for d in tmp_decks:
                self.decks.append(d)
    
    def loadAllUsedCards(self):
        if len(self.cards) == 0:
            self.loadAllDecks()
            for i in self.decks:
                for j in i:
                    if "(ORI)" not in j:
                        self.cards.append(j)
            self.cards = sorted(list(set(self.cards)))
    
    def downloadAllCardsData(self):
        URL = 'https://mtgjson.com/json/AllCards.json.zip'
        url = urlopen(URL)
        output = open('zippedCards.zip', 'wb')
        output.write(url.read())
        output.close()
        os.system('unzip zippedCards.zip')
        os.remove('zippedCards.zip')
        self.allcards = pd.read_json('AllCards.json')
    
    def loadAllCardsData(self):
        if os.path.isfile('AllCards.json') == False:
            self.downloadAllCardsData()
        if len(self.allcards) == 0:
            self.allcards = pd.read_json('AllCards.json')
            
    def loadCurrentCubeCards(self, filename):
        self.cube = []
        self.cube_dict = {}
        my_file = open('data/'+filename, 'r')
        my_string = my_file.read()
        my_list = my_string.split('\r\n')
        for i in my_list:
            tmp = i.split(' ')
            if len(tmp) > 1:
                cardname = " ".join(tmp[1:])
                self.cube.append(cardname)
                self.cube_dict[tmp[0]] = cardname
            
    def findInvalidCardnames(self, cubefile):
        self.invalid_names = []
        self.loadAllCardsData()
        self.loadAllUsedCards()
        self.loadCurrentCubeCards(cubefile)
        for i in self.cards:
            if i not in self.allcards:
                self.invalid_names.append(i)
        for i in self.cube:
            if i not in self.allcards:
                self.invalid_names.append(i)
        
        print self.invalid_names
                
    def correctNamesInAllDecks(self):
        self.loadAllDeckFilenames()
        self.findInvalidCardnames()
        for deckfile in self.deckfiles:
            my_file = open('data/decks/'+deckfile, 'r')
            tmp = my_file.read()
            for j in self.invalid_names:
                tmp = tmp.replace(j, cards_to_correct[j])
            my_file.close()
            out_file = open('data/decks/'+deckfile, 'w')
            out_file.write(tmp)
            out_file.close()
            
    def loadPlayerAliases(self):
        if len(self.alias_dict) == 0:
            my_file = open('data/player_alias.txt', 'r')
            my_string = my_file.read()
            my_list = my_string.split('\n')
            for i in my_list:
                if i != '':
                    tmp = i.split(' ')
                    self.alias_dict[tmp[0]] = tmp[1] 
        
        return self.alias_dict
    
    def loadMatchesFromTxt(self, filename):
        my_file = open('data/'+filename, 'r')
        my_string = my_file.read()
        my_string = my_string.replace('\r', '')
        my_list = my_string.split('\n')
        loaded_matches = []
        for i in my_list:
            if i != '':
                tmp = i.split(' ')
                loaded_matches.append(tmp)
        return loaded_matches
    
    def loadMatches(self, filename):
        matches = pd.read_json('data/matches/'+filename)
        match_data = []
        for i in matches["matches"]: 
            record = []
            record.append({"player": i[0]["player"], "score": i[0]["score"]})
            record.append({"player": i[1]["player"], "score": i[1]["score"]})
            match_data.append(record)
        return match_data
    
    def loadDecksFromTxt(self, filename):
        my_file = open('data/'+filename, 'r')
        my_string = my_file.read()
        my_string = my_string.replace('\r', '')
        my_list = my_string.split('NEXT_DECK')
        decks = []
        for i in my_list:
            current_deck = {}
            tmp = i.split('\n')
            for j in tmp:
                dict_line = j.split(':')
                if dict_line[0] == 'player':
                    current_deck['player'] = dict_line[1]
                if dict_line[0] == 'name': 
                    current_deck['name'] = dict_line[1]
                if dict_line[0] == 'cards':
                    card_list = dict_line[1].split(' ')
                    current_deck['cards'] = card_list
                if dict_line[0] == 'basic':
                    basic_list = dict_line[1].split(' ')
                    current_deck['basic'] = basic_list
            decks.append(current_deck)
        return decks
                     

loader = Loader()

loader.loadAllUsedCards()

cubefile = 'numbered_18_3.txt'

#loader.loadCurrentCubeCards(cubefile)

decks = loader.loadDecksFromTxt('decklists_18_3.txt')

print decks

#print loader.cube_dict

#loader.findInvalidCardnames(cubefile)

#mdata = loader.loadMatches('matches_18_3.json')
#print mdata
     

[{'cards': ['394', '64', '30', '2', '450', '47', '167', '15', '10', '4', '49', '493', '529', '68', '31', '24', '27', '408', '151', '412', '517', '40', '358', '407', '150', '159', '410', '369', ''], 'player': 'Kuba', 'name': 'WB Mid-Range', 'basic': ['6', '0', '6', '0', '0']}, {'cards': ['91', '182', '99', '197', '486', '134', '363', '74', '535', '103', '201', '170', '516', '155', '181', '144', '500', '75', '172', '173', '190', '184', '188', '537', '158'], 'player': 'TomekP', 'name': 'UB Control', 'basic': ['0', '5', '10', '0', '0']}, {'cards': ['282', '310', '111', '102', '333', '294', '96', '322', '514', '119', '505', '295', '123', '112', '98', '327', '390', '431', '437', '488', '308', '338', '86', '81', '87', '433'], 'player': 'Wielki', 'name': 'UG Ramp', 'basic': ['0', '7', '0', '0', '7']}, {'cards': ['149', '312', '210', '421', '198', '420', '524', '207', '427', '326', '194', '171', '148', '417', '307', '157', '306', '419', '185', '315', '467', '350', '154', '161', '297'], 'player'

In [20]:
class Saver():
    def __init__(self):
        saver_exists = True
        pass
    
    def saveMatchesAsJson(self, matches_string, filename):
        my_file = open('data/matches/'+filename+'.json', 'w')
        my_file.write(matches_string)
        my_file.close()
        
    def saveMatchScoresForCoverage(self, prepared, filename):
        ftmp = filename.split('.')
        my_file = open('data/coverage/'+ftmp[0]+'.txt', 'w')
        my_file.write(prepared)
        my_file.close()
        

saver = Saver()

In [22]:
class Converter():
    def __init__(self):
        converter_exists = True
        if loader_exists == False:
            loader = Loader()
        if saver_exists == False:
            saver = Saver()
            
    def getNumberOfRounds(self, match_num):
        for i in range (1, match_num/2):
            if (i*(i+1))/2 == match_num:
                return i+1
    
    def convertMatchesToJson(self, filename):
        alias_dict = loader.loadPlayerAliases()
        matches_to_convert = loader.loadMatchesFromTxt(filename)
        ftmp = filename.split('.')
        draft_num = ftmp[0].replace('matches_', '')
        matches_json = '{\n'
        matches_json +='\t"draft":"'+draft_num+'",\n'
        matches_json +='\t"cube_version":"'+draft_num+'",\n'
        matches_json +='\t"matches":[\n'
        for i in range(0, len(matches_to_convert)):
            matches_json+= '\t\t[{"player":"'+alias_dict[matches_to_convert[i][0]]+'", "score":'+matches_to_convert[i][1]
            matches_json+= '}, {"player":"'+alias_dict[matches_to_convert[i][2]]+'", "score":'+matches_to_convert[i][3]
            if i < len(matches_to_convert)-1:
                matches_json+='}],\n'
            else: matches_json+='}]\n\t]\n'
        matches_json+='}'
        
        saver.saveMatchesAsJson(matches_json, ftmp[0])
        
    def prepareMatchesForCoverage(self, filename):
        match_data = loader.loadMatches(filename)
        match_num = len(match_data)
        draft_rounds = self.getNumberOfRounds(match_num)
        prepared = ''
        current_round = 0
        match_per_round = match_num / draft_rounds
        for i in range(0, match_num):
            if i%match_per_round == 0:
                current_round +=1
                prepared+='\nRunda #'+str(current_round)+'\n\n'
            prepared+=str(i+1)+'. '+match_data[i][0]['player']+' vs '+match_data[i][1]['player']+' '
            prepared+=str(match_data[i][0]['score'])+':'+str(match_data[i][1]['score'])+'\n'
        
        saver.saveMatchScoresForCoverage(prepared, filename)
        
    def convertDecksTxtToJson(self, filename):
        ftmp = filename.split('_')
        draft_num = ftmp[1]+'_'+ftmp[2].replace('.txt', '')
        
        
converter = Converter()
#converter.convertMatchesToJson('matches_18_3.txt')
converter.prepareMatchesForCoverage('matches_18_3.json')

**Data Saver**